In [2]:
import urllib
import pandas as pd
import json

In [16]:
# url = 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=200000'

# with urllib.request.urlopen (url) as req:
#     df = pd.DataFrame(json.loads(req.read())['result']['records'])
# df.to_csv("Resale HDB Prices.csv", index=False)

In [54]:
df = pd.read_csv("Resale HDB Prices.csv")
df['month'] = pd.to_datetime(df['month'])
df


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01-01,61 years 04 months,1979,10 TO 12,1,406
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01-01,60 years 07 months,1978,01 TO 03,2,108
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01-01,62 years 05 months,1980,01 TO 03,3,602
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01-01,62 years 01 month,1980,04 TO 06,4,465
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01-01,62 years 05 months,1980,01 TO 03,5,601
...,...,...,...,...,...,...,...,...,...,...,...,...
155413,YISHUN,5 ROOM,Model A,133.0,YISHUN ST 22,610000.0,2023-06-01,67 years 11 months,1992,01 TO 03,155414,291
155414,YISHUN,5 ROOM,3Gen,120.0,YISHUN ST 51,715000.0,2023-06-01,93 years 09 months,2018,07 TO 09,155415,513B
155415,YISHUN,5 ROOM,Improved,113.0,YISHUN ST 51,645000.0,2023-06-01,93 years 03 months,2017,10 TO 12,155416,511B
155416,YISHUN,EXECUTIVE,Apartment,143.0,YISHUN AVE 2,845000.0,2023-06-01,63 years 04 months,1987,04 TO 06,155417,791


In [72]:
def getResaleHDBPrices():
    df = pd.read_csv("Resale HDB Prices.csv")[:140000]
    notOverlapping = True
    offset = 0
    res = []
    step = 10000
    while (notOverlapping):
        url = f'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit={step}&sort=month%20desc&offset={offset}'
        
        with urllib.request.urlopen (url) as req:
            res += json.loads(req.read())['result']['records']
        #print(type(res[-1]['_id']))
        if res[-1]['_id'] <= len(df):
            notOverlapping = False
        offset += step
    df2 = pd.DataFrame(res).sort_values(by=['_id'])
    df2 = df2[df2['_id'] > len(df)].reset_index(drop=True)

    return pd.concat([df, df2], ignore_index=True, sort=False).drop_duplicates(ignore_index=True)

In [73]:
df3 = getResaleHDBPrices()


In [74]:
df3

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,61 years 04 months,1979,10 TO 12,1,406
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,60 years 07 months,1978,01 TO 03,2,108
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,62 years 05 months,1980,01 TO 03,3,602
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,62 years 01 month,1980,04 TO 06,4,465
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,62 years 05 months,1980,01 TO 03,5,601
...,...,...,...,...,...,...,...,...,...,...,...,...
155413,YISHUN,5 ROOM,Model A,133,YISHUN ST 22,610000,2023-06,67 years 11 months,1992,01 TO 03,155414,291
155414,YISHUN,5 ROOM,3Gen,120,YISHUN ST 51,715000,2023-06,93 years 09 months,2018,07 TO 09,155415,513B
155415,YISHUN,5 ROOM,Improved,113,YISHUN ST 51,645000,2023-06,93 years 03 months,2017,10 TO 12,155416,511B
155416,YISHUN,EXECUTIVE,Apartment,143,YISHUN AVE 2,845000,2023-06,63 years 04 months,1987,04 TO 06,155417,791


In [38]:
len(df)

155418

In [18]:
pd.to_datetime('2017-01')

Timestamp('2017-01-01 00:00:00')